# Smart Device Automation
Now that we have generated a complete set of APIs and web pages for PhotovoltaicMeasurement data, now we will build an OpenShift CustomResourceDefinition (CRD), Ansible Role, and Ansible Playbooks to easily deploy PhotovoltaicMeasurement smart devices, register the devices in the IoT Agent, update the entity data in the Context Broker, and send the same entity data to the Smart Village Platform so that we can visualize it. 

## Install prerequisite operator-sdk

Run the commands below to deploy the Operator SDK, for building new CRDs like PhotovoltaicMeasurement into our `smartvillage-operator``. 

In [ ]:
export ARCH=$(case $(uname -m) in x86_64) echo -n amd64 ;; aarch64) echo -n arm64 ;; *) echo -n $(uname -m) ;; esac)
export OS=$(uname | awk '{print tolower($0)}')
export OPERATOR_SDK_DL_URL=https://github.com/operator-framework/operator-sdk/releases/download/v1.33.0
curl -o /tmp/operator-sdk_${OS}_${ARCH} -LO ${OPERATOR_SDK_DL_URL}/operator-sdk_${OS}_${ARCH}
gpg --keyserver keyserver.ubuntu.com --recv-keys 052996E2A20B5C7E
curl -LO ${OPERATOR_SDK_DL_URL}/checksums.txt
curl -LO ${OPERATOR_SDK_DL_URL}/checksums.txt.asc
gpg -u "Operator SDK (release) <cncf-operator-sdk@cncf.io>" --verify checksums.txt.asc
grep operator-sdk_${OS}_${ARCH} checksums.txt | sha256sum -c -
chmod +x /tmp/operator-sdk_${OS}_${ARCH} && mv /tmp/operator-sdk_${OS}_${ARCH} ~/.local/bin/operator-sdk


## Install python dependencies
An IoT Agent expects Entity Data from all Smart Data Models to be sent through a message broker like RabbitMQ. We install `pika` so that messages can be sent to RabbitMQ to register new PhotovoltaicMeasurement Smart Devices and update Smart Device entity data. 

In [ ]:
pip install pika

## Reconfigure NGSI-LD Smart Village Sync
When an IoT Agent receives a message from RabbitMQ with identifiers that match a registered smart device, it sends the data to the Context Broker in NGSI-LD format. After the Context Broker updates the entity data for the device in PostgreSQL, it looks if any services are subscribed to the entity. We configure all smart devices with a Context Broker subscription (webhook) so that the Context Broker sends all entity data updates to the NGSI-LD Smart Village Sync application we deployed in the project. The NGSI-LD Smart Village Sync application is configured to send subscription data straight to the Smart Village Platform API. We need to update our NGSI-LD Smart Village Sync application to point to the new Smart Village application we started running in our OpenShift AI Workbench for development. 

In [ ]:
oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  set env deployment/ngsild-smartvillage-sync SMARTVILLAGE_HOST_NAME=$(oc get route/vscode-devel -o jsonpath={.spec.host})
oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  set env deployment/ngsild-smartvillage-sync SMARTVILLAGE_PORT=443
oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  set env deployment/ngsild-smartvillage-sync SMARTVILLAGE_SSL=true
echo DONE

In [ ]:
(cd ~/smartvillage-operator && operator-sdk create api --group smartvillage --version v1 --kind PhotovoltaicMeasurement --generate-role)
echo DONE

In [ ]:
(cd ~/smartvillage-operator && ansible-playbook write-smart-data-model-templates.yaml -e ENTITY_TYPE=PhotovoltaicMeasurement)
echo DONE

In [ ]:
echo "
apiVersion: smartvillage.computate.org/v1
kind: PhotovoltaicMeasurement
metadata:
  name: star
  namespace: $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
spec:
  iotagent:
    base_url: http://iotagent-json:4041
  context_broker:
    base_url: http://scorpiobroker:9090
  ngsi_ld:
    service: smarttrafficlights
    service_path: /Sweden/Veberod/CityCenter
    context: https://raw.githubusercontent.com/computate-org/smartabyar-smartvillage-static/main/fiware/context.jsonld
  device:
    id: star
    subscription_url: http://ngsild-smartvillage-sync:8080
  message_broker:
    transport: AMQP
    host: default-rabbitmq
    port: 5672
    user: user
    secret:
      name: rabbitmq-password
      key: rabbitmq-password
  smartvillage:
    auth_secret_name: smartvillage
    auth_token_url: https://sso.smartabyarsmartvillage.org/auth/realms/SMARTVILLAGE/protocol/openid-connect/token
    site_base_url: https://$(oc get route/vscode-devel -o jsonpath={.spec.host})
    smart_model_api: /api/PhotovoltaicMeasurement
  attributes:
    areaServed: {
        "coordinates": [
          [
            [
              13.492267031569327,
              55.63382607412092
            ],
            [
              13.492145868594037,
              55.63382081304013
            ],
            [
              13.492234410768901,
              55.633773463278374
            ],
            [
              13.492201789967083,
              55.63370506907751
            ],
            [
              13.492304312485658,
              55.63373926619286
            ],
            [
              13.492397514773728,
              55.63370506907751
            ],
            [
              13.492369554087844,
              55.63377609382226
            ],
            [
              13.492439455803208,
              55.63382081304013
            ],
            [
              13.492350913629707,
              55.63382081304013
            ],
            [
              13.492304312485658,
              55.633878684892835
            ],
            [
              13.492267031569327,
              55.63382607412092
            ]
          ]
        ],
        "type": "Polygon"
      }
    name: star
" | tee /tmp/PhotovoltaicMeasurement-star.yaml >/dev/null
echo DONE

In [ ]:
ansible-playbook ~/smartvillage-operator/apply-photovoltaicmeasurement.yaml \
  -e ansible_operator_meta_namespace=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e crd_path=/tmp/PhotovoltaicMeasurement-star.yaml
echo DONE